In [8]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

In [3]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

#fo.delete_dataset("multimodal-shapes-subset")
dataset = load_from_hub("maxspeer/assessment2_spheres_and_cube_2k_2",
                         name="multimodal-shapes-subset",
                         num_workers=1,
                         batch_size=500,
                        #max_samples=3000,
                           overwrite=True
                        )

/Users/maximilian/miniforge3/envs/compv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset
Importing samples...
 100% |███████████████| 6000/6000 [47.9ms elapsed, 0s remaining, 125.1K samples/s]  


In [4]:
from pathlib import Path
import sys
project_root = Path("..").resolve()
sys.path.append(str(project_root))

In [5]:
from src.datasets import MultimodalDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


IMG_SIZE = 64

img_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1] TODO correct non deprecated version
])

BATCH_SIZE = 32
train_dataset = MultimodalDataset(dataset,"train",img_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)


val_dataset = MultimodalDataset(dataset,"val",img_transforms)
valid_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [6]:
from src.models import IntermediateFusionNet, LateFusionModel
# Fusion models to compare
fusionModels = [
    ("concat_strided", lambda: IntermediateFusionNet(4,4, "concat", embedder="strided")),
    ("add_strided", lambda: IntermediateFusionNet(4,4, "add", embedder="strided")),
    ("hadamard_strided", lambda: IntermediateFusionNet(4,4, "hadamard", embedder="strided")),
    ("late_strided", lambda: LateFusionModel(embedder="strided")),
]

In [7]:
from src.training import getWandbRun, train_model, get_outputs_fusion

epochs = 3

batch_size = 16
lr = 0.001

for mode, modelBlueprint in fusionModels:
    with getWandbRun(mode, "rgb/lidar", batch_size=batch_size, epochs=epochs) as run:
        model = modelBlueprint().to(device)

        print(f"Training fusion mode: {mode}")
        model_param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Number of trainable parameters: {model_param_count}")
        run.config.update({"number_of_parameters": model_param_count})

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                        shuffle=True, drop_last=True)
        valid_dataloader = DataLoader(val_dataset, batch_size=batch_size,
                                        shuffle=True, drop_last=True)

        opt = Adam(model.parameters(), lr=lr)
        loss_func = nn.BCEWithLogitsLoss()
        train_loss, valid_loss = train_model(model=model,
                                            optimizer=opt,
                                            train_dataloader=train_dataloader,
                                            valid_dataloader=valid_dataloader,
                                            loss_func=loss_func,
                                            get_outputs=get_outputs_fusion,
                                            wandbrun=run,
                                            epochs=epochs)
        run.finish()

wandb: Currently logged in as: maximilian-speer (maximilian-speer-hasso-plattner-institut) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb.init fertig


Traceback (most recent call last):
  File "/var/folders/28/mjr4t5694n99_vdxd2rbvdf80000gn/T/ipykernel_31089/4290814467.py", line 10, in <module>
    model = modelBlueprint().to(device)
            ^^^^^^^^^^^^^^^^
  File "/var/folders/28/mjr4t5694n99_vdxd2rbvdf80000gn/T/ipykernel_31089/1063334327.py", line 4, in <lambda>
    ("concat_strided", lambda: IntermediateFusionNet(4,4, "concat", embedder="strided")),
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/maximilian/code/assessment2/src/models.py", line 60, in __init__
    self.relu = F.Identity
                ^^^^^^^^^^
AttributeError: module 'torch.nn.functional' has no attribute 'Identity'


AttributeError: module 'torch.nn.functional' has no attribute 'Identity'